In [1]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals
# text in Western (Windows 1252)

import pickle
import numpy as np
# import StringIO
import math
from keras import optimizers, metrics
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras import regularizers
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.constraints import maxnorm
from keras.layers import Flatten
from keras.optimizers import SGD
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)

Using Theano backend.


In [2]:
# %run ../../../prepare_data.py

import sys
sys.path.insert(0, '../../../')
from prepare_data import *

In [14]:
# %run ../../../prepare_data.py
# X_train, X_other_features_train, y_train, X_test, X_other_features_test, y_test, X_validate, X_other_features_validate, y_validate = generate_full_matrix_inputs('../../internal_representations/inputs/content_shuffle_vector.h5', '../../internal_representations/inputs/shuffle_vector')
# save_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', X_train, y_train, other_features = X_other_features_train)
# save_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', X_test, y_test, other_features = X_other_features_test)
# save_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', X_validate, y_validate,  other_features = X_other_features_validate)
# X_train, X_other_features_train, y_train = load_inputs('../../internal_representations/inputs/shuffled_letters_train.h5', other_features=True)
# X_test, X_other_features_test, y_test = load_inputs('../../internal_representations/inputs/shuffled_letters_test.h5', other_features=True)
# X_validate, X_other_features_validate, y_validate = load_inputs('../../internal_representations/inputs/shuffled_letters_validate.h5', other_features=True)

data = Data('l', save_generated_data=False, additional_letter_attributes=False, reverse_inputs=False, accent_classification=True)
data.generate_data('TEST_train',
                   'TEST_test',
                   'TEST_validate')

READING CONTENT...
CONTENT READ SUCCESSFULLY
CREATING DICTIONARY...
DICTIONARY CREATION SUCCESSFUL!
GENERATING X AND y...
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
SHUFFELING INPUTS...
INPUTS SHUFFELED!
GENERATION SUCCESSFUL!


In [4]:
gen2 = data.generator('train', 16)
# test1 = next(gen1)
test2 = next(gen2)

In [7]:
pos = 0
print(len(feature_dictionary))
print(np.array(test2[0][0]).shape)
print(np.array(test2[0][1]).shape)
# for el in test2[0][0]:
# #     print(el)
#     print (data.decode_x(el, dictionary))
#     print(data.decode_x_other_features(feature_dictionary, [el]))

# for el in test2[1]:
#     print(el)

10
(16, 23, 36)
(16, 140)


In [12]:
# print (X_train.shape)
# print (X_test.shape)
# print (X_validate.shape)
pos = 0
# print (decode_input(X_train[pos], dictionary))
# print (decode_X_features(feature_dictionary, [X_other_features_train[pos]]))
# print(decode_position(y_train[pos]))
# print('------------------------------------')
print (data.x_train.shape)
print (data.x_test.shape)
print (data.x_validate.shape)
# pos = 2
print (data.decode_x(data.x_train[pos], dictionary))
print (data.decode_x_other_features(feature_dictionary, [data.x_other_features_train[pos]]))
print(data.x_other_features_train[pos])
print(data.decode_y(data.y_train[pos]))

(430151, 23, 36)
(52058, 23, 36)
(54222, 23, 36)
ašjenlautkajan
Agsnpn-
Agsnpn-
[1 1 0 0 0 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 6]
[2, 5]


In [17]:
num_examples = len(data.x_train) # training set size
nn_output_dim = 13
nn_hdim = 516
batch_size = 16
actual_epoch = 40
num_fake_epoch = 20



# conv_input_shape=(23, 36)
conv_input_shape=(23, 30)
othr_input = (150, )

conv_input = Input(shape=conv_input_shape, name='conv_input')
x_conv = Conv1D(115, (3), padding='same', activation='relu')(conv_input)
x_conv = Conv1D(46, (3), padding='same', activation='relu')(x_conv)
x_conv = MaxPooling1D(pool_size=2)(x_conv)
x_conv = Flatten()(x_conv)
# x_conv = Dense(516, activation='relu', kernel_constraint=maxnorm(3))(x_conv)

othr_input = Input(shape=othr_input, name='othr_input')
# x_othr = Dense(256, input_dim=167, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(512, activation='relu')(othr_input)
# x_othr = Dropout(0.3)(x_othr)
# x_othr = Dense(256, activation='relu')(othr_input)

x = concatenate([x_conv, othr_input])
# x = Dense(1024, input_dim=(516 + 256), activation='relu')(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.3)(x)
x = Dense(nn_output_dim, activation='sigmoid')(x)




model = Model(inputs=[conv_input, othr_input], outputs=x)
opt = optimizers.Adam(lr=1E-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=[actual_accuracy,])
# model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [18]:
history = model.fit_generator(data.generator('train', batch_size), 
                              data.x_train.shape[0]/(batch_size * num_fake_epoch), 
                              epochs=actual_epoch*num_fake_epoch, 
                              validation_data=data.generator('test', batch_size), 
                              validation_steps=data.x_test.shape[0]/(batch_size * num_fake_epoch),
                              verbose=2)

Epoch 1/800
21s - loss: 0.2277 - actual_accuracy: 0.0492 - val_loss: 0.1782 - val_actual_accuracy: 0.1695
Epoch 2/800
21s - loss: 0.1681 - actual_accuracy: 0.2592 - val_loss: 0.1391 - val_actual_accuracy: 0.4532
Epoch 3/800
21s - loss: 0.1349 - actual_accuracy: 0.4625 - val_loss: 0.1193 - val_actual_accuracy: 0.5303
Epoch 4/800
21s - loss: 0.1118 - actual_accuracy: 0.5566 - val_loss: 0.0939 - val_actual_accuracy: 0.6411
Epoch 5/800
20s - loss: 0.0861 - actual_accuracy: 0.6836 - val_loss: 0.0739 - val_actual_accuracy: 0.7715
Epoch 6/800
21s - loss: 0.0687 - actual_accuracy: 0.7679 - val_loss: 0.0590 - val_actual_accuracy: 0.8313
Epoch 7/800
24s - loss: 0.0604 - actual_accuracy: 0.8057 - val_loss: 0.0523 - val_actual_accuracy: 0.8501
Epoch 8/800
21s - loss: 0.0544 - actual_accuracy: 0.8302 - val_loss: 0.0479 - val_actual_accuracy: 0.8700
Epoch 9/800
22s - loss: 0.0515 - actual_accuracy: 0.8378 - val_loss: 0.0450 - val_actual_accuracy: 0.8704
Epoch 10/800
22s - loss: 0.0461 - actual_accur

25s - loss: 0.0051 - actual_accuracy: 0.9843 - val_loss: 0.0420 - val_actual_accuracy: 0.9398
Epoch 79/800
23s - loss: 0.0059 - actual_accuracy: 0.9827 - val_loss: 0.0377 - val_actual_accuracy: 0.9479
Epoch 80/800
23s - loss: 0.0057 - actual_accuracy: 0.9826 - val_loss: 0.0355 - val_actual_accuracy: 0.9502
Epoch 81/800
24s - loss: 0.0053 - actual_accuracy: 0.9845 - val_loss: 0.0370 - val_actual_accuracy: 0.9444
Epoch 82/800
24s - loss: 0.0053 - actual_accuracy: 0.9853 - val_loss: 0.0356 - val_actual_accuracy: 0.9455
Epoch 83/800
26s - loss: 0.0051 - actual_accuracy: 0.9857 - val_loss: 0.0419 - val_actual_accuracy: 0.9433
Epoch 84/800
24s - loss: 0.0051 - actual_accuracy: 0.9855 - val_loss: 0.0371 - val_actual_accuracy: 0.9448
Epoch 85/800
25s - loss: 0.0052 - actual_accuracy: 0.9845 - val_loss: 0.0439 - val_actual_accuracy: 0.9360
Epoch 86/800
24s - loss: 0.0051 - actual_accuracy: 0.9848 - val_loss: 0.0405 - val_actual_accuracy: 0.9433
Epoch 87/800
24s - loss: 0.0052 - actual_accuracy:

24s - loss: 0.0035 - actual_accuracy: 0.9901 - val_loss: 0.0369 - val_actual_accuracy: 0.9571
Epoch 156/800
22s - loss: 0.0033 - actual_accuracy: 0.9908 - val_loss: 0.0482 - val_actual_accuracy: 0.9479
Epoch 157/800
25s - loss: 0.0030 - actual_accuracy: 0.9917 - val_loss: 0.0389 - val_actual_accuracy: 0.9513
Epoch 158/800
25s - loss: 0.0034 - actual_accuracy: 0.9914 - val_loss: 0.0444 - val_actual_accuracy: 0.9452
Epoch 159/800
24s - loss: 0.0027 - actual_accuracy: 0.9928 - val_loss: 0.0477 - val_actual_accuracy: 0.9425
Epoch 160/800
25s - loss: 0.0033 - actual_accuracy: 0.9913 - val_loss: 0.0499 - val_actual_accuracy: 0.9417
Epoch 161/800
23s - loss: 0.0032 - actual_accuracy: 0.9917 - val_loss: 0.0379 - val_actual_accuracy: 0.9486
Epoch 162/800
23s - loss: 0.0033 - actual_accuracy: 0.9914 - val_loss: 0.0372 - val_actual_accuracy: 0.9536
Epoch 163/800
24s - loss: 0.0025 - actual_accuracy: 0.9926 - val_loss: 0.0441 - val_actual_accuracy: 0.9486
Epoch 164/800
26s - loss: 0.0028 - actual_

25s - loss: 0.0022 - actual_accuracy: 0.9943 - val_loss: 0.0413 - val_actual_accuracy: 0.9532
Epoch 232/800
24s - loss: 0.0022 - actual_accuracy: 0.9941 - val_loss: 0.0444 - val_actual_accuracy: 0.9494
Epoch 233/800
26s - loss: 0.0024 - actual_accuracy: 0.9933 - val_loss: 0.0380 - val_actual_accuracy: 0.9540
Epoch 234/800
25s - loss: 0.0024 - actual_accuracy: 0.9940 - val_loss: 0.0514 - val_actual_accuracy: 0.9444
Epoch 235/800
22s - loss: 0.0022 - actual_accuracy: 0.9944 - val_loss: 0.0497 - val_actual_accuracy: 0.9479
Epoch 236/800
22s - loss: 0.0021 - actual_accuracy: 0.9942 - val_loss: 0.0398 - val_actual_accuracy: 0.9594
Epoch 237/800
23s - loss: 0.0026 - actual_accuracy: 0.9928 - val_loss: 0.0439 - val_actual_accuracy: 0.9548
Epoch 238/800
23s - loss: 0.0023 - actual_accuracy: 0.9937 - val_loss: 0.0543 - val_actual_accuracy: 0.9429
Epoch 239/800
23s - loss: 0.0022 - actual_accuracy: 0.9943 - val_loss: 0.0421 - val_actual_accuracy: 0.9444
Epoch 240/800
22s - loss: 0.0021 - actual_

24s - loss: 0.0024 - actual_accuracy: 0.9936 - val_loss: 0.0465 - val_actual_accuracy: 0.9540
Epoch 308/800
25s - loss: 0.0019 - actual_accuracy: 0.9951 - val_loss: 0.0466 - val_actual_accuracy: 0.9494
Epoch 309/800
23s - loss: 0.0023 - actual_accuracy: 0.9947 - val_loss: 0.0463 - val_actual_accuracy: 0.9521
Epoch 310/800
23s - loss: 0.0021 - actual_accuracy: 0.9947 - val_loss: 0.0525 - val_actual_accuracy: 0.9471
Epoch 311/800
24s - loss: 0.0021 - actual_accuracy: 0.9948 - val_loss: 0.0494 - val_actual_accuracy: 0.9448
Epoch 312/800
27s - loss: 0.0021 - actual_accuracy: 0.9945 - val_loss: 0.0406 - val_actual_accuracy: 0.9559
Epoch 313/800
24s - loss: 0.0024 - actual_accuracy: 0.9934 - val_loss: 0.0356 - val_actual_accuracy: 0.9555
Epoch 314/800
23s - loss: 0.0018 - actual_accuracy: 0.9954 - val_loss: 0.0328 - val_actual_accuracy: 0.9590
Epoch 315/800
25s - loss: 0.0018 - actual_accuracy: 0.9957 - val_loss: 0.0380 - val_actual_accuracy: 0.9475
Epoch 316/800
25s - loss: 0.0020 - actual_

...

25s - loss: 0.0021 - actual_accuracy: 0.9945 - val_loss: 0.0414 - val_actual_accuracy: 0.9590
Epoch 460/800
27s - loss: 0.0023 - actual_accuracy: 0.9947 - val_loss: 0.0474 - val_actual_accuracy: 0.9528
Epoch 461/800
25s - loss: 0.0015 - actual_accuracy: 0.9960 - val_loss: 0.0490 - val_actual_accuracy: 0.9525
Epoch 462/800
25s - loss: 0.0018 - actual_accuracy: 0.9954 - val_loss: 0.0474 - val_actual_accuracy: 0.9479
Epoch 463/800
26s - loss: 0.0020 - actual_accuracy: 0.9950 - val_loss: 0.0390 - val_actual_accuracy: 0.9559
Epoch 464/800
26s - loss: 0.0017 - actual_accuracy: 0.9961 - val_loss: 0.0408 - val_actual_accuracy: 0.9563
Epoch 465/800
25s - loss: 0.0020 - actual_accuracy: 0.9955 - val_loss: 0.0416 - val_actual_accuracy: 0.9498
Epoch 466/800
25s - loss: 0.0015 - actual_accuracy: 0.9961 - val_loss: 0.0497 - val_actual_accuracy: 0.9509
Epoch 467/800
25s - loss: 0.0015 - actual_accuracy: 0.9959 - val_loss: 0.0417 - val_actual_accuracy: 0.9593
Epoch 468/800
26s - loss: 0.0021 - actual_

26s - loss: 0.0023 - actual_accuracy: 0.9942 - val_loss: 0.0464 - val_actual_accuracy: 0.9563
Epoch 536/800
26s - loss: 0.0021 - actual_accuracy: 0.9944 - val_loss: 0.0440 - val_actual_accuracy: 0.9578
Epoch 537/800
27s - loss: 0.0015 - actual_accuracy: 0.9965 - val_loss: 0.0327 - val_actual_accuracy: 0.9620
Epoch 538/800
25s - loss: 0.0017 - actual_accuracy: 0.9958 - val_loss: 0.0484 - val_actual_accuracy: 0.9459
Epoch 539/800
26s - loss: 0.0020 - actual_accuracy: 0.9952 - val_loss: 0.0437 - val_actual_accuracy: 0.9521
Epoch 540/800
25s - loss: 0.0018 - actual_accuracy: 0.9956 - val_loss: 0.0447 - val_actual_accuracy: 0.9555
Epoch 541/800
26s - loss: 0.0012 - actual_accuracy: 0.9969 - val_loss: 0.0496 - val_actual_accuracy: 0.9525
Epoch 542/800
26s - loss: 0.0017 - actual_accuracy: 0.9960 - val_loss: 0.0541 - val_actual_accuracy: 0.9463
Epoch 543/800
26s - loss: 0.0019 - actual_accuracy: 0.9955 - val_loss: 0.0508 - val_actual_accuracy: 0.9513
Epoch 544/800
25s - loss: 0.0019 - actual_

25s - loss: 0.0018 - actual_accuracy: 0.9953 - val_loss: 0.0552 - val_actual_accuracy: 0.9486
Epoch 612/800
25s - loss: 0.0016 - actual_accuracy: 0.9967 - val_loss: 0.0533 - val_actual_accuracy: 0.9452
Epoch 613/800
26s - loss: 0.0019 - actual_accuracy: 0.9959 - val_loss: 0.0529 - val_actual_accuracy: 0.9502
Epoch 614/800
27s - loss: 0.0019 - actual_accuracy: 0.9950 - val_loss: 0.0479 - val_actual_accuracy: 0.9502
Epoch 615/800
24s - loss: 0.0018 - actual_accuracy: 0.9955 - val_loss: 0.0508 - val_actual_accuracy: 0.9463
Epoch 616/800
25s - loss: 0.0013 - actual_accuracy: 0.9964 - val_loss: 0.0415 - val_actual_accuracy: 0.9555
Epoch 617/800
25s - loss: 0.0014 - actual_accuracy: 0.9964 - val_loss: 0.0414 - val_actual_accuracy: 0.9582
Epoch 618/800
25s - loss: 0.0013 - actual_accuracy: 0.9962 - val_loss: 0.0371 - val_actual_accuracy: 0.9666
Epoch 619/800
27s - loss: 0.0019 - actual_accuracy: 0.9956 - val_loss: 0.0527 - val_actual_accuracy: 0.9479
Epoch 620/800
26s - loss: 0.0021 - actual_

29s - loss: 0.0015 - actual_accuracy: 0.9961 - val_loss: 0.0448 - val_actual_accuracy: 0.9563
Epoch 688/800
25s - loss: 0.0015 - actual_accuracy: 0.9962 - val_loss: 0.0468 - val_actual_accuracy: 0.9528
Epoch 689/800
26s - loss: 0.0021 - actual_accuracy: 0.9954 - val_loss: 0.0552 - val_actual_accuracy: 0.9502
Epoch 690/800
27s - loss: 0.0020 - actual_accuracy: 0.9954 - val_loss: 0.0465 - val_actual_accuracy: 0.9555
Epoch 691/800
26s - loss: 0.0012 - actual_accuracy: 0.9969 - val_loss: 0.0477 - val_actual_accuracy: 0.9517
Epoch 692/800
28s - loss: 0.0018 - actual_accuracy: 0.9960 - val_loss: 0.0499 - val_actual_accuracy: 0.9555
Epoch 693/800
27s - loss: 0.0022 - actual_accuracy: 0.9954 - val_loss: 0.0589 - val_actual_accuracy: 0.9448
Epoch 694/800
25s - loss: 0.0016 - actual_accuracy: 0.9955 - val_loss: 0.0530 - val_actual_accuracy: 0.9505
Epoch 695/800
26s - loss: 0.0014 - actual_accuracy: 0.9963 - val_loss: 0.0522 - val_actual_accuracy: 0.9498
Epoch 696/800
28s - loss: 0.0017 - actual_

25s - loss: 0.0017 - actual_accuracy: 0.9957 - val_loss: 0.0509 - val_actual_accuracy: 0.9525
Epoch 764/800
27s - loss: 0.0014 - actual_accuracy: 0.9959 - val_loss: 0.0481 - val_actual_accuracy: 0.9559
Epoch 765/800
25s - loss: 0.0015 - actual_accuracy: 0.9964 - val_loss: 0.0551 - val_actual_accuracy: 0.9544
Epoch 766/800
26s - loss: 0.0014 - actual_accuracy: 0.9964 - val_loss: 0.0551 - val_actual_accuracy: 0.9505
Epoch 767/800
25s - loss: 0.0019 - actual_accuracy: 0.9954 - val_loss: 0.0461 - val_actual_accuracy: 0.9540
Epoch 768/800
28s - loss: 0.0012 - actual_accuracy: 0.9967 - val_loss: 0.0564 - val_actual_accuracy: 0.9513
Epoch 769/800
26s - loss: 0.0018 - actual_accuracy: 0.9959 - val_loss: 0.0458 - val_actual_accuracy: 0.9532
Epoch 770/800
26s - loss: 0.0016 - actual_accuracy: 0.9961 - val_loss: 0.0550 - val_actual_accuracy: 0.9505
Epoch 771/800
25s - loss: 0.0017 - actual_accuracy: 0.9957 - val_loss: 0.0458 - val_actual_accuracy: 0.9543
Epoch 772/800
25s - loss: 0.0018 - actual_

In [20]:
name = '40_epoch'
model.save(name + '.h5')
output = open(name + '_history.pkl', 'wb')
pickle.dump(history.history, output)
output.close()

In [7]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()